#### Creating Table named DimCustomer  

In [10]:
CREATE TABLE dbo.DimCustomer
(
    CustomerId INT NOT NULL,
    [Name]     VARCHAR(60) NOT NULL,
    [City]     VARCHAR(40) NOT NULL,
    [State]    VARCHAR(40) NOT NULL,
    Segment    VARCHAR(20) NOT NULL
);

#### Inserting Data Into DimCustomer Table

In [11]:
INSERT INTO dbo.DimCustomer (CustomerId, [Name], [City], [State], Segment) VALUES
(101,'Aarav Shah','Mumbai','Maharashtra','Retail'),
(102,'Neha Verma','Pune','Maharashtra','Retail'),
(103,'Rohan Mehta','Delhi','Delhi','Enterprise'),
(104,'Isha Gupta','Bengaluru','Karnataka','Retail'),
(105,'Vikram Singh','Hyderabad','Telangana','Enterprise'),
(106,'Priya Nair','Kochi','Kerala','Retail'),
(107,'Kunal Joshi','Jaipur','Rajasthan','Retail'),
(108,'Ananya Das','Kolkata','West Bengal','Enterprise'),
(109,'Rahul Jain','Ahmedabad','Gujarat','Retail'),
(110,'Simran Kaur','Chandigarh','Chandigarh','Enterprise'),
(111,'Mohit Kapoor','Noida','Uttar Pradesh','Retail'),
(112,'Divya Iyer','Chennai','Tamil Nadu','Enterprise');

#### Select Data from DimCustomer Table

In [12]:
SELECT * FROM DimCustomer

#### Create Table named FactSales

In [3]:
CREATE TABLE dbo.FactSales
(
    SalesId      INT         NOT NULL,
    CustomerId   INT         NULL,          -- keep nullable to demo IS NULL cases if needed
    [SalesDate]  DATE        NOT NULL,
    Product      VARCHAR(40) NOT NULL,
    Qty          INT         NOT NULL,
    UnitPrice    DECIMAL(18,2) NOT NULL,
    DiscountPct  DECIMAL(5,2)  NOT NULL      -- e.g., 5.00 = 5%
);

#### Inserting Data Into FactSales Table

In [4]:
INSERT INTO dbo.FactSales (SalesId, CustomerId, [SalesDate], Product, Qty, UnitPrice, DiscountPct) VALUES
(1,  101, '2025-08-01', 'Fabric Pro',      2,  3500.00, 5.00),
(2,  102, '2025-08-01', 'Power BI Plus',   1,  1800.00, 0.00),
(3,  105, '2025-08-02', 'Fabric Pro',      3,  3500.00, 10.00),
(4,  109, '2025-08-03', 'Data Gateway',    5,   400.00, 0.00),
(5,  110, '2025-08-03', 'Fabric Pro',      1,  3500.00, 0.00),
(6,  112, '2025-08-04', 'PBIRS',           2,  2200.00, 5.00),
(7,  103, '2025-08-04', 'Power BI Plus',   4,  1800.00, 0.00),
(8,  108, '2025-08-05', 'OneLake Add-on',  6,   250.00, 0.00),
(9,  107, '2025-08-06', 'Fabric Pro',      1,  3500.00, 0.00),
(10, 113, '2025-08-06', 'Power BI Plus',   2,  1800.00, 0.00),  -- 113 is MISSING in Dim
(11, 999, '2025-08-07', 'Fabric Pro',      1,  3500.00, 15.00), -- 999 is MISSING in Dim
(12, 101, '2025-08-07', 'OneLake Add-on', 10,   250.00, 0.00),
(13, 105, '2025-08-08', 'Data Gateway',    2,   400.00, 0.00),
(14, 104, '2025-08-08', 'Fabric Pro',      2,  3500.00, 0.00);

#### Getting rows from FactSales Table

In [5]:
SELECT * FROM FactSales

#### Calculating Line Amount For Each Sale

In [13]:
SELECT
    s.SalesId,
    s.CustomerId,
    s.[SalesDate],
    s.Product,
    s.Qty,
    s.UnitPrice,
    s.DiscountPct,
    CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2)) AS LineAmount
FROM dbo.FactSales AS s;

#### Retrieving the Sales Transactions with the Customer Details by Using INNER JOIN and Calculating Line Amount

In [14]:
SELECT
    s.SalesId, s.CustomerId, c.[Name], c.[City], c.[State], c.Segment,
    s.Product, s.Qty,
    CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2)) AS LineAmount
FROM dbo.FactSales AS s
INNER JOIN dbo.DimCustomer AS c
    ON s.CustomerId = c.CustomerId;

#### Retrieving the Sales Transactions without the Customer Details by Using LEFT JOIN and Calculating Line Amount

#### 

In [40]:
SELECT
    s.SalesId, s.CustomerId, c.[Name], c.[City], c.[State], c.Segment,
    s.Product, s.Qty,
    CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2)) AS LineAmount
FROM dbo.FactSales AS s
LEFT JOIN dbo.DimCustomer AS c
    ON s.CustomerId = c.CustomerId
WHERE c.CustomerId IS NULL;

#### Retrieving Sales with CustomerInfo and line Amount using Inner Joins

In [17]:
SELECT
    s.SalesId, s.CustomerId, c.[Name], c.[City], c.[State], c.Segment,
    s.Product, s.Qty,
    CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2)) AS LineAmount
FROM dbo.FactSales AS s
INNER JOIN dbo.DimCustomer AS c
    ON s.CustomerId = c.CustomerId;

#### Retrieving Sales and CustomerInfo by Using the Full Outer Join

In [18]:
SELECT
    s.SalesId, s.CustomerId AS FactCustomerId,
    c.CustomerId AS DimCustomerId, c.[Name], c.[City], c.[State], c.Segment,
    s.Product, s.Qty
FROM dbo.FactSales AS s
FULL OUTER JOIN dbo.DimCustomer AS c
    ON s.CustomerId = c.CustomerId
ORDER BY
    CASE WHEN s.CustomerId IS NULL THEN 1 ELSE 0 END,  -- dims with no facts first
    COALESCE(s.CustomerId, c.CustomerId), s.SalesId;

In [19]:
SELECT s.SalesId, s.CustomerId, s.SalesDate, s.Product, s.Qty
FROM dbo.FactSales AS s
WHERE s.SalesDate > '2025-08-03'
AND s.Product = 'Fabric Pro';

In [39]:
SELECT s.SalesId, s.CustomerId, s.SalesDate, s.Product, s.Qty
FROM dbo.FactSales AS s
WHERE s.SalesDate = '2025-08-03'
AND s.Product = 'Fabric Pro';

In [33]:
SELECT COALESCE(c.CustomerId, s.CustomerId) AS CustomerId, 
c.[Name], SUM(CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2))) AS TotalSales
FROM dbo.FactSales AS s
LEFT JOIN dbo.DimCustomer AS c
ON s.CustomerId = c.CustomerId
GROUP BY COALESCE(c.CustomerId, s.CustomerId), c.[Name]
ORDER BY TotalSales DESC;

In [29]:
SELECT COALESCE(c.CustomerId, s.CustomerId) AS CustomerId, 
c.[Name], SUM(CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2))) AS TotalSales
FROM dbo.FactSales AS s  
INNER JOIN dbo.DimCustomer AS c
ON s.CustomerId = c.CustomerId
GROUP BY COALESCE(c.CustomerId, s.CustomerId), c.[Name]
ORDER BY TotalSales DESC;

In [34]:
SELECT COALESCE(c.CustomerId, s.CustomerId) AS CustomerId, 
c.[Name], SUM(CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2))) AS TotalSales
FROM dbo.FactSales AS s  
RIGHT JOIN dbo.DimCustomer AS c
ON s.CustomerId = c.CustomerId
GROUP BY COALESCE(c.CustomerId, s.CustomerId), c.[Name]
ORDER BY TotalSales DESC;

In [35]:
SELECT COALESCE(c.CustomerId, s.CustomerId) AS CustomerId, 
c.[Name], SUM(CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct/100.0) AS DECIMAL(18,2))) AS TotalSales
FROM dbo.FactSales AS s  
FULL OUTER JOIN dbo.DimCustomer AS c
ON s.CustomerId = c.CustomerId
GROUP BY COALESCE(c.CustomerId, s.CustomerId), c.[Name]
ORDER BY TotalSales DESC;

In [36]:
SELECT 
    c.CustomerId AS CustomerId, 
    c.[Name], 
    s.SalesId,
    CAST(s.Qty * s.UnitPrice * (1 - s.DiscountPct / 100.0) AS DECIMAL(18, 2)) AS RowTotal
FROM dbo.FactSales AS s
CROSS JOIN dbo.DimCustomer AS c

In [32]:
INSERT INTO dbo.FactSales (SalesId, CustomerId, [SalesDate], Product, Qty, UnitPrice, DiscountPct) 
VALUES(15,  101, '2025-08-01', 'Fabric Pro', 2,  3500.00, 5.00);

In [21]:
SELECT GETDATE()

In [26]:
SELECT * FROM dbo.FactSales OPTION (FOR TIMESTAMP AS OF '2025-08-18T02:34:48.73')